In [43]:
import  pandas as pd
import string
import pickle
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [57]:
data = pd.read_csv('emails.csv')
data

,Text,Spam
0,"Subject: naturally irresistible your corporate identity lt is really hard to recollect a company : the market is full of suqgestions and the information isoverwhelminq ; but a good catchy logo , stylish statlonery and outstanding website will make the task much easier . we do not promise that havinq ordered a iogo your company will automaticaily become a world ieader : it isguite ciear that without good products , effective business organization and practicable aim it will be hotat nowadays market ; but we do promise that your marketing efforts will become much more effective . here is the list of clear benefits : creativeness : hand - made , original logos , specially done to reflect your distinctive company image . convenience : logo and stationery are provided in all formats ; easy - to - use content management system letsyou change your website content and even its structure . promptness : you will see logo drafts within three business days . affordability : your marketing break - through shouldn ' t make gaps in your budget . 100 % satisfaction guaranteed : we provide unlimited amount of changes with no extra fees for you to be surethat you will love the result of this collaboration . have a look at our portfolio _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ not interested . . . _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _",1
1,Subject: the stock trading gunslinger fanny is merrill but muzo not colza attainder and penultimate like esmark perspicuous ramble is segovia not group try slung kansas tanzania yes chameleon or continuant clothesman no libretto is chesapeake but tight not waterway herald and hawthorn like chisel morristown superior is deoxyribonucleic not clockwork try hall incredible mcdougall yes hepburn or einsteinian earmark no sapling is boar but duane not plain palfrey and inflexible like huzzah pepperoni bedtime is nameable not attire try edt chronography optima yes pirogue or diffusion albeit no,1
2,"Subject: unbelievable new homes made easy im wanting to show you this homeowner you have been pre - approved for a $ 454 , 169 home loan at a 3 . 72 fixed rate . this offer is being extended to you unconditionally and your credit is in no way a factor . to take advantage of this limited time opportunity all we ask is that you visit our website and complete the 1 minute post approval form look foward to hearing from you , dorcas pittman",1
3,"Subject: 4 color printing special request additional information now ! click here click here for a printable version of our order form ( pdf format ) phone : ( 626 ) 338 - 8090 fax : ( 626 ) 338 - 8102 e - mail : ramsey @ goldengraphix . com request additional information now ! click here click here for a printable version of our order form ( pdf format ) golden graphix & printing 5110 azusa canyon rd . irwindale , ca 91706 this e - mail message is an advertisement and / or solicitation .",1
4,"Subject: do not have money , get software cds from here ! software compatibility . . . . ain ' t it great ? grow old along with me the best is yet to be . all tradgedies are finish ' d by death . all comedies are ended by marriage .",1
...,...,...
5723,"Subject: re : research and development charges to gpg here it is ! - - - - - - - - - - - - - - - - - - - - - - forwarded by shirley crenshaw / hou / ect on 08 / 14 / 2000 07 : 47 am - - - - - - - - - - - - - - - - - - - - - - - - - - - vince j kaminski 08 / 10 / 2000 02 : 25 pm to : vera apodaca / et & s / enron @ enron cc : vince j kaminski / hou / ect @ ect , shirley crenshaw / hou / ect @ ect , pinnamaneni krishnarao / hou / ect @ ect subject : re : research and development charges to gpg vera , we shall talk to the accounting group about the correction . vince 08 / 09 / 2000 03 : 26 pm vera apodaca @ enron vera apodaca @ enron vera apodaca @ enron 08 / 09 / 2000 03 : 26 pm 08 / 09 / 2000 03 : 26 pm to

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    5728 non-null   object
 1   Spam    5728 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 89.6+ KB


In [11]:
data['Spam'].unique()

array([1, 0])

In [12]:
def process_text(text):
    words_without_punctuation = [char for char in text if char not in string.punctuation]
    words_without_punctuation = ''.join(words_without_punctuation)
    
    return ' '.join([word for word in words_without_punctuation.split() if word.lower() not in stopwords.words('english')])

In [13]:
data['Text'] = data['Text'].apply(process_text)

In [14]:
stemmer = PorterStemmer()
def stemming(text):
    return ''.join([stemmer.stem(word) for word in text])

data['Text'] = data['Text'].apply(stemming)

In [15]:
data

,Text,Spam
0,subject naturally irresistible corporate ident...,1
1,subject stock trading gunslinger fanny merrill...,1
2,subject unbelievable new homes made easy im wa...,1
3,subject 4 color printing special request addit...,1
4,subject money get software cds software compat...,1
...,...,...
5723,subject research development charges gpg forwa...,0
5724,subject receipts visit jim thanks invitation v...,0
5725,subject enron case study update wow day super ...,0
5726,subject interest david please call shirley cre...,0


In [16]:
vectorizer = CountVectorizer()
message_bagofwords = vectorizer.fit_transform(data['Text'])
message_bagofwords.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [19]:
x_train, x_test, y_train, y_test = train_test_split(message_bagofwords, data['Spam'], random_state=42, test_size=0.2)

In [20]:
print("Shape of x_train: ", x_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of x_test: ", x_test.shape)
print("Shape of y_test: ", y_test.shape)

Shape of x_train:  (4582, 37187)
Shape of y_train:  (4582,)
Shape of x_test:  (1146, 37187)
Shape of y_test:  (1146,)


In [21]:
svm = SVC()
svm.fit(x_train, y_train)

SVC()

In [22]:
y_pred = svm.predict(x_test)

In [31]:
report = classification_report(y_test, y_pred,output_dict=True)
report_df = pd.DataFrame(report).transpose()

In [32]:
report_df

,precision,recall,f1-score,support
0,0.971526,0.996495,0.983852,856.000000
1,0.988806,0.913793,0.949821,290.000000
accuracy,0.975567,0.975567,0.975567,0.975567
macro avg,0.980166,0.955144,0.966837,1146.000000
weighted avg,0.975899,0.975567,0.975241,1146.000000


In [34]:
with open("svm_model.pkl", "wb") as f:
    pickle.dump(svm, f)

In [36]:
nb = MultinomialNB()
nb.fit(x_train, y_train)

MultinomialNB()

In [37]:
y_pred_nb = nb.predict(x_test)

In [38]:
report_nb = classification_report(y_test, y_pred_nb,output_dict=True)
report_nb_df = pd.DataFrame(report_nb).transpose()

In [39]:
report_nb_df

,precision,recall,f1-score,support
0,0.997647,0.990654,0.994138,856.000000
1,0.972973,0.993103,0.982935,290.000000
accuracy,0.991274,0.991274,0.991274,0.991274
macro avg,0.985310,0.991879,0.988537,1146.000000
weighted avg,0.991403,0.991274,0.991303,1146.000000


In [42]:
with open("nb_model.pkl", "wb") as f2:
    pickle.dump(nb, f2)

In [44]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

LogisticRegression()

In [45]:
y_pred_lr = lr.predict(x_test)

In [46]:
report_lr = classification_report(y_test, y_pred_lr,output_dict=True)
report_lr_df = pd.DataFrame(report_lr).transpose()

In [47]:
report_lr_df

,precision,recall,f1-score,support
0,0.993007,0.995327,0.994166,856.000000
1,0.986111,0.979310,0.982699,290.000000
accuracy,0.991274,0.991274,0.991274,0.991274
macro avg,0.989559,0.987319,0.988432,1146.000000
weighted avg,0.991262,0.991274,0.991264,1146.000000


In [48]:
with open("lr_model.pkl", "wb") as f3:
    pickle.dump(lr, f3)

In [50]:
with open('count_vectorizer.pkl', 'wb') as cv:
    pickle.dump(vectorizer, cv)